# Getting N of police that would be disciplined under NYPD standards

In [82]:
import pandas as pd
from selenium import webdriver
from pathlib import Path
from tools import tree
from datetime import datetime as dt

today = dt.today().strftime("%Y-%m-%d")

In [83]:
today

'2020-06-12'

In [3]:
PROCESSED_DATA = Path("../data/processed/")

In [5]:
tree(PROCESSED_DATA)

+ ../data/processed
        + force-nj-com-12-Jun-20-checkpoint.csv
    + force-nj-com-12-Jun-20.csv


In [6]:
data = pd.read_csv(PROCESSED_DATA / 'force-nj-com-12-jun-20.csv')

In [9]:
test_url = data['full_url'][0]

In [10]:
browser = webdriver.Chrome(executable_path='./chromedriver')

In [11]:
browser.get(test_url)

In [15]:
important_numbers = browser.find_elements_by_class_name("important_num_red2")

In [43]:
cleaned_ = []
for element in important_numbers:
    print(element.text.replace("'s", "").replace("'", ""))

Los Angeles
4
New York City
15
Chicago
16


In [45]:
[element.text.replace("'s", "").replace("'", "") for element in important_numbers]

['Los Angeles', '4', 'New York City', '15', 'Chicago', '16']

In [19]:
data.head()

,dept_name,relative_url,full_url,likelihood_force_by_pop,likelihood_force_by_arrests,no_data_found
0,"Aberdeen, Monmouth",/database/pd-dept/aberdeen-monmouth,http://force.nj.com/database/pd-dept/aberdeen-...,294%,42%,NaN
1,"Absecon, Atlantic",/database/pd-dept/absecon-atlantic,http://force.nj.com/database/pd-dept/absecon-a...,583%,68%,NaN
2,"Allendale, Bergen",/database/pd-dept/allendale-bergen,http://force.nj.com/database/pd-dept/allendale...,NaN,8%,NaN
3,"Allenhurst, Monmouth",/database/pd-dept/allenhurst-monmouth,http://force.nj.com/database/pd-dept/allenhurs...,NaN,NaN,NaN
4,"Allentown, Monmouth",/database/pd-dept/allentown-monmouth,http://force.nj.com/database/pd-dept/allentown...,NaN,NaN,NaN


In [47]:
def get_url_important_numbers(url):
    browser.get(url)
    important_numbers = browser.find_elements_by_class_name("important_num_red2")
    return [element.text.replace("'s", "").replace("'", "") for element in important_numbers]

In [37]:
tupes = []
for row in data[['dept_name', 'full_url']].itertuples():
    dept = {}
    dept['name'] = row.dept_name
    dept['full_url'] = row.full_url
    tupes.append(dept)

In [48]:
for dept in tupes:
    dept['important_numbers'] = get_url_important_numbers(dept['full_url'])

In [60]:
for dept in tupes:
    if len(dept['important_numbers']) == 6:
        first_city = dept['important_numbers'][0]
        first_city_n = dept['important_numbers'][1]
        second_city = dept['important_numbers'][2]
        second_city_n = dept['important_numbers'][3]
        third_city = dept['important_numbers'][4]
        third_city_n = dept['important_numbers'][5]
        
        dept[f"{first_city}_n"] = first_city_n
        dept[f"{second_city}_n"] = second_city_n
        dept[f"{third_city}_n"] = third_city_n
        

Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles New York City Chicago
Los Angeles Ne

In [63]:
working_df = pd.DataFrame(tupes)

In [67]:
working_df = working_df[['name', 'full_url', 'Los Angeles_n', 'New York City_n', 'Chicago_n']]

In [68]:
working_df.columns = ['name', 'full_url', 'LA_numbers', 'NYC_numbers', 'Chicago_numbers']

In [69]:
data.head()

,dept_name,relative_url,full_url,likelihood_force_by_pop,likelihood_force_by_arrests,no_data_found
0,"Aberdeen, Monmouth",/database/pd-dept/aberdeen-monmouth,http://force.nj.com/database/pd-dept/aberdeen-...,294%,42%,NaN
1,"Absecon, Atlantic",/database/pd-dept/absecon-atlantic,http://force.nj.com/database/pd-dept/absecon-a...,583%,68%,NaN
2,"Allendale, Bergen",/database/pd-dept/allendale-bergen,http://force.nj.com/database/pd-dept/allendale...,NaN,8%,NaN
3,"Allenhurst, Monmouth",/database/pd-dept/allenhurst-monmouth,http://force.nj.com/database/pd-dept/allenhurs...,NaN,NaN,NaN
4,"Allentown, Monmouth",/database/pd-dept/allentown-monmouth,http://force.nj.com/database/pd-dept/allentown...,NaN,NaN,NaN


In [74]:
final_df = data.join(working_df, rsuffix='_r')

In [75]:
final_df.to_csv(PROCESSED_DATA / 'draft-city_n.csv', index = False, encoding = 'utf-8')

In [78]:
voi = [
    'dept_name', 
    'relative_url', 
    'full_url', 
    'likelihood_force_by_pop',
    'likelihood_force_by_arrests', 
    'no_data_found', 
    'LA_numbers', 
    'NYC_numbers', 
    'Chicago_numbers'
]

In [80]:
final_df[voi].head()

,dept_name,relative_url,full_url,likelihood_force_by_pop,likelihood_force_by_arrests,no_data_found,LA_numbers,NYC_numbers,Chicago_numbers
0,"Aberdeen, Monmouth",/database/pd-dept/aberdeen-monmouth,http://force.nj.com/database/pd-dept/aberdeen-...,294%,42%,NaN,4,15,16
1,"Absecon, Atlantic",/database/pd-dept/absecon-atlantic,http://force.nj.com/database/pd-dept/absecon-a...,583%,68%,NaN,0,9,13
2,"Allendale, Bergen",/database/pd-dept/allendale-bergen,http://force.nj.com/database/pd-dept/allendale...,NaN,8%,NaN,0,2,3
3,"Allenhurst, Monmouth",/database/pd-dept/allenhurst-monmouth,http://force.nj.com/database/pd-dept/allenhurs...,NaN,NaN,NaN,0,4,4
4,"Allentown, Monmouth",/database/pd-dept/allentown-monmouth,http://force.nj.com/database/pd-dept/allentown...,NaN,NaN,NaN,0,1,1


In [84]:
final_df[voi].to_csv(PROCESSED_DATA / f'force-nj-com-v2-{today}.csv', encoding = 'utf-8', index = False)